Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr

import sys
sys.path.append('../src')
import utils as ut

In [ ]:
#year0 = 1851
year0 = 1951

year1 = year0+149

In [ ]:
ds = xr.open_dataset(f'../data/alldata_{year0}_{year1}.nc')
ds = ds.isel(exp=slice(0,5))

#Extract few models for quick calcs
#ds = ds.isel(esm=slice(0,2),ism=slice(0,2))

In [ ]:
for b,bmp in enumerate(ut.bmps):
    slr_nf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    slr_wf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    for s,ss in enumerate(ds.ssp.values):
        for e,es in enumerate(ds.esm.values):
            for i,ii in enumerate(ds.ism.values):
                TMP,IML,SLR = ut.iterate(ds,ism=ii,esm=es,ssp=ss,bmp=bmp)
                slr_nf[s,e,i,:] = np.sum(SLR[0,:,:],axis=1)
                slr_wf[s,e,i,:] = np.sum(SLR[-1,:,:],axis=1)
                print(f'{bmp:4s}',ss,f'{es:14s}',f'{ii:10s}',f'{slr_nf[s,e,i,-1]: .3f}',f'{slr_wf[s,e,i,-1]: .3f}')       
    #Save
    slr_nf2 = xr.DataArray(slr_nf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise without feedback'})
    slr_wf2 = xr.DataArray(slr_wf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise with feedback'})

    ds2 = xr.Dataset({'slr_nf':slr_nf2,'slr_wf':slr_wf2})
    ds2.to_netcdf(f'../data/ensemble_{bmp}_{year0}_{year1}.nc',mode='w')
    ds2.close()
    print(f'Saved {bmp}')
ds.close()